In [1]:
#get image urls
from os import walk
import pandas as pd
#import random

def fetchURLfrom_csvFolder(path): # You need oganize these URLs
    files = []
    for (dirpath, dirnames, filenames) in walk(path):
        for name in filenames:
            if '.csv' in name:
                p = '../data/'+name
                files.append(p)
        break
    url_set = set()
    for f in files:
        df = pd.read_csv(f)
        url_set = url_set | set(df['image'])
    url_list = list(url_set)
    print('Obtained {} URLs'.format(len(url_list)))
    #random.shuffle(url_list)
    return url_list

url_list = fetchURLfrom_csvFolder('../data/')


#******************************************************************************************************#

import threading
import time
import requests
API_End = 'https://focdujc0w8.execute-api.us-west-2.amazonaws.com/Stage0/image_url_analyze?img_url='

def cleanJSON_print(jsonContent):
    keyLen = 0
    for k in jsonContent:
        keyLen = max(len(k), keyLen)
    for k in jsonContent:
        print ('{}{}: {}'.format(k,  ' '*(keyLen - len(k)), jsonContent[k]))

def analyze_from_API(URL):
    newURL = API_End + URL.replace('&', '|')
    response = None
    URL_score = 0 # ------------------> 
    try:
        response = requests.get(newURL)
    except Exception as e:
        print('Error: {}'.format(e))
        return
    if response and response.status_code == 200:
        jsonContent = response.json()
        ############### Implement Here ###############
        # Calculate URL Image score from json format "jsonContent"
        
        ##############################################
        
    else:
        print('Response status code: {}'.format(response.status_code))
    return URL_score # return score- ---------------------->
        

class myThread(threading.Thread):
    def __init__(self, threadID, URL):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = 'Thread-' + str(threadID)
        self.url = URL
    def run(self):
        print("Starting <{}>\n".format(self.name))
        #threadLock.acquire()
        score = analyze_from_API(self.url) # Obtain score here !!!!! -------------------->
        global globalNums
        globalNums -= 1
        print("Ended <{}> at time ({})\n".format(self.name, time.ctime(time.time()) ))
        
        #threadLock.release()

# Example to create new threads
# thread1 = myThread(1)
# thread2 = myThread(2)

# # Start new Threads
# thread1.start()
# thread2.start()

Obtained 5146 URLs


In [3]:
#threadLock = threading.Lock()
global globalNums
globalNums = 5
threads = [0]*globalNums
for i, url in enumerate(url_list[0:globalNums]):
    threads[i] = myThread(i+1, url)
for t in threads:
    t.start()
for t in threads:
    t.join()

if globalNums <= 0:
    print('Main Thread Completed')

Starting <Thread-1>

Starting <Thread-2>

Starting <Thread-3>
Starting <Thread-4>
Starting <Thread-5>



Ended <Thread-4> at time (Tue Feb  4 16:24:47 2020)

Ended <Thread-3> at time (Tue Feb  4 16:24:47 2020)

Ended <Thread-1> at time (Tue Feb  4 16:24:47 2020)

Ended <Thread-2> at time (Tue Feb  4 16:24:47 2020)

Ended <Thread-5> at time (Tue Feb  4 16:24:48 2020)

Main Thread Completed
